- Data su zo stranky www.soga.sk, jedna sa o vsetky aukcie od roku 2004 az po 2024, cca 100 aukcii a 21,000 drazenych diel

In [1]:
import sqlite3
import pandas as pd
import pandasql as ps
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
from bokeh.io import output_notebook

In [3]:
# Connect to the SQLite database
conn = sqlite3.connect('soga.db')

# Query to select data from the database
query = """
SELECT 
    UPPER(SUBSTR(autor, 1, INSTR(autor, '(') - 1)) AS autor,
    a.nazov, 
    vydrazene,
    CAST(REPLACE(konecna_cena, ' ', '') AS INTEGER) AS konecna_cena,
    CAST(REPLACE(vyvolavacia_cena, ' ', '') AS INTEGER) AS vyvolavacia_cena,
    rok,
    technika,
    datum,
    url_soga
FROM diela a 
LEFT JOIN aukcie b ON a.id_aukcie = b.id 
WHERE TRUE 
-- AND autor like '%FILKO%'
ORDER BY CAST(REPLACE(konecna_cena, ' ', '') AS INTEGER) DESC;
"""

# Load data into a Pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
df.head()

DatabaseError: Execution failed on sql '
SELECT 
    UPPER(SUBSTR(autor, 1, INSTR(autor, '(') - 1)) AS autor,
    a.nazov, 
    vydrazene,
    CAST(REPLACE(konecna_cena, ' ', '') AS INTEGER) AS konecna_cena,
    CAST(REPLACE(vyvolavacia_cena, ' ', '') AS INTEGER) AS vyvolavacia_cena,
    rok,
    technika,
    datum,
    url_soga
FROM diela a 
LEFT JOIN aukcie b ON a.id_aukcie = b.id 
WHERE TRUE 
-- AND autor like '%FILKO%'
ORDER BY CAST(REPLACE(konecna_cena, ' ', '') AS INTEGER) DESC;
': no such table: diela

# Priprava dat

In [ ]:
from unidecode import unidecode  # Import the unidecode function
import re


# Custom function to standardize and reorder names
def standardize_and_order_name(name):
    # Convert accented characters to ASCII equivalents
    name = unidecode(name)
    name= name.upper()

    # Split the name into parts
    name_parts = name.split()

    # Reorder the name parts alphabetically
    standardized_name = ' '.join(sorted(name_parts))
    
    

    return standardized_name


def extract_first_year(date_string):
    # Regular expression pattern to match a year
    year_pattern = re.compile(r'\b\d{4}\b')

    # Find the first match in the string
    match = year_pattern.search(date_string)

    # Extract and return the matched year
    if match:
        return int(match.group())

    return None  # Return None if no year is found


In [ ]:
df['autor']=df['autor'].apply(standardize_and_order_name)

In [ ]:
df['olej']=df['technika'].apply(lambda x: 'olej' in x.lower())

In [ ]:
df['rok_vytvorenia']=df['rok'].apply(lambda x: extract_first_year(x)).fillna(-1).astype(int).replace(-1,None)
df['rok_drazby']=df['datum'].apply(lambda x: extract_first_year(x)).fillna(-1).astype(int).replace(-1,None)

In [ ]:
df.head()

# Analyza

## Top 50

### Top 50 najdrazsich autorov podla priemernej ceny

In [ ]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_vydrazenych_od_2004, 
ROUND(avg(konecna_cena)) as priemerna_vydrazena_cena_EUR,
min(konecna_cena)as minimalna_vydrazena_cena_EUR,
max(konecna_cena)as maximalna_vydrazena_cena_EUR
--sum(konecna_cena)as celkovo_vydrazene_EUR
from df 
where True 
and konecna_cena is not null 
--and olej
group by 1 
having count(*)>5
order by 3
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]

### Top 50 najdrazsich autorov - oleje

In [ ]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_vydrazenych_od_2004, 
ROUND(avg(konecna_cena)) as priemerna_vydrazena_cena_EUR,
min(konecna_cena)as minimalna_vydrazena_cena_EUR,
max(konecna_cena)as maximalna_vydrazena_cena_EUR
--sum(konecna_cena)as celkovo_vydrazene_EUR
from df 
where True 
and konecna_cena is not null 
and olej
group by 1 
having count(*)>5
order by 3
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]

### Top 50 najdrazsich diel

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and olej
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

### Top 50 najdrazsich diel - nie oleje

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
and not olej
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

### Top 50 najdrazsich diel - Pasteka

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%PASTEKA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

### Top 50 najdrazsich diel - Binder

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%BINDER %'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

### Top 50 najdrazsich diel - Filko

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FILKO%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

### Top 50 najdrazsich diel - Laluha

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%LALUHA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

### Top 50 najdrazsich diel - Benka

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%BENKA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

### Top 50 najdrazsich diel - Fulla

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FULLA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

### Top 50 najdrazsich diel - Klimo

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%KLIMO%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

### Top 50 najdrazsich diel - Roller

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%ROLLER%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

### Top 50 najdrazsich diel - Fila

In [ ]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FILA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

## Najvyssi narast a pokles cien

### Najvyssi narast

In [ ]:
from scipy.stats import linregress
pd.set_option('display.max_colwidth', None)

# Calculate the average price for each name in each year
df_avg = df.groupby(['autor', 'rok_drazby'])['konecna_cena'].agg(['count', 'mean']).reset_index()
df_avg = df_avg[df_avg['count']>=3][['autor', 'rok_drazby', 'mean']]

# Filter authors with at least 50 rows
author_counts = df_avg['autor'].value_counts()
selected_authors = author_counts[author_counts >= 5].index
df_avg_filtered = df_avg[df_avg['autor'].isin(selected_authors)]

# Calculate the slope for each author
slopes = []

for author in selected_authors:
    author_data = df_avg_filtered[df_avg_filtered['autor'] == author]
    slope, _, _, _, _ = linregress(author_data['rok_drazby'], author_data['mean'])
    slopes.append({'autor': author, 'priemerny_rocny_narast_v_cene': slope})

# Create a DataFrame for the slopes
slopes_df = pd.DataFrame(slopes)

# Display the slopes
slopes_df[slopes_df['priemerny_rocny_narast_v_cene'].notna()].sort_values('priemerny_rocny_narast_v_cene', ascending=False).reset_index(drop=True)[0:20]

In [ ]:
slopes_df[slopes_df['priemerny_rocny_narast_v_cene'].notna()].sort_values('priemerny_rocny_narast_v_cene', ascending=True).reset_index(drop=True)[0:20]

In [ ]:
# Calculate the average price for each name in each year
df_avg = df.groupby(['autor', 'rok_drazby'])['konecna_cena'].mean().reset_index()

# Filter authors with at least 50 rows
selected_authors = ['MILAN PASTEKA', 'FILKO STANO', 'LALUHA MILAN']
df_avg_filtered = df_avg[df_avg['autor'].isin(selected_authors)]

# Create Bokeh ColumnDataSource
source = ColumnDataSource(df_avg_filtered)

# Plotting
output_notebook()
p = figure(plot_width=800, plot_height=400, title='Average Price Trends Over Time for Each Name',
           x_axis_label='Year', y_axis_label='Average Price')

# Add a line for each autor
for color, autor in zip(Category10[6], df_avg_filtered['autor'].unique()):
    autor_data = df_avg_filtered[df_avg_filtered['autor'] == autor]
    autor_source = ColumnDataSource(autor_data)
    p.line(x='rok_drazby', y='konecna_cena', source=autor_source, line_width=2, color=color, legend_label=autor)
    p.circle(x='rok_drazby', y='konecna_cena', source=autor_source, size=8, color=color, legend_label=autor)

p.legend.location = 'top_left'
show(p)

## Najvyssie percento vydrazenych

In [ ]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_drazenych_od_2004, 
sum(vydrazene) as pocet_vydrazenych_od_2004,
round(sum(vydrazene)*100.0/count(*),2) as percento_vydrazenych
from df 
where True 
--and konecna_cena is not null 
--and olej
group by 1 
having count(*)>50
order by 4
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]